In [1]:
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score,mean_squared_error, log_loss
import plotly.express as px
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
import numpy as np

In [2]:
data = load_wine(as_frame=True)
X,Y = data.data,data.target
nfeats = len(X.columns)

In [3]:
# Select only classes 0 and 1
X = X[(Y==0) | (Y==1)].reset_index(drop=True)
Y = Y[(Y==0) | (Y==1)].reset_index(drop=True).values
Y[Y==0]=-1


In [4]:
def loss_func(x, y, weights):
    return np.sum(np.log(1 + np.exp(-y * (x.dot(weights)))))

def grad_update(
    x, y, weights, feats_selection=[], mode="all", learn_rate=0.1, curr_coord=0
):
    assert weights.shape == (nfeats + 1, 1)

    if x.shape != (nfeats + 1, 1) and mode != "all":
        x = x.reshape([-1, 1])

    if mode == "all":
        gradient = -np.sum((y * x) / (1 + np.exp(y * (x.dot(weights)))), 0).reshape(
            [-1, 1]
        )
    else:
        gradient = -(y * x) / (1 + np.exp(y * (weights.T.dot(x)))).reshape([-1, 1])

    if feats_selection == "random":
        temp = np.zeros(gradient.shape)
        random_coord = np.random.choice(14)
        temp[random_coord] = gradient[random_coord]
        gradient = temp

    elif feats_selection == "max":
        t = np.argmax(abs(gradient))
        gradient[gradient != gradient[t]] = 0

    elif feats_selection == "round_robin":
        temp = np.zeros(gradient.shape)
        temp[curr_coord] = gradient[curr_coord]
        gradient = temp

    return weights - (learn_rate * gradient.reshape([-1, 1]))


In [ ]:
runs = 100
nfeats = len(X.columns)
epochs = 500
fin_loss = []

for i in range(runs):
    curr_coord = 1
    fig = go.Figure()

    X_train, y_train = X.copy(), Y.copy()

    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)

    X_train = np.append(X_train, np.ones([len(X_train), 1]), 1)
    y_train = y_train.reshape([-1, 1])

    weights = np.random.random([nfeats + 1, 1])
    weightsRandomUpdate = weights.copy()
    weightsMaxGradUpdate = weights.copy()
    weightsRR = weights.copy()

    loss = []
    lossMaxGradUpdate = []
    lossRandomUpdate = []
    lossRR = []

    for e in range(epochs):
        for it in range(len(X_train)):

            # All weights update
            weights = grad_update(X_train[it], y_train[it], weights, mode="stochastic")
            curr_loss = loss_func(X_train, y_train, weights)

            # Random weights update
            weightsRandomUpdate = grad_update(
                X_train[it],
                y_train[it],
                weightsRandomUpdate,
                feats_selection="random",
                mode="stochastic",
            )
            curr_loss_rand_update = loss_func(X_train, y_train, weightsRandomUpdate)

            # Only Maximum Gradient Weight Update
            weightsMaxGradUpdate = grad_update(
                X_train[it],
                y_train[it],
                weightsMaxGradUpdate,
                feats_selection="max",
                mode="stochastic",
            )
            curr_loss_max_grad = loss_func(X_train, y_train, weightsMaxGradUpdate)

            # Round Robin Weight Update
            weightsRR = grad_update(
                X_train[it],
                y_train[it],
                weightsRR,
                mode="stochastic",
                feats_selection="round_robin",
                curr_coord=curr_coord,
            )
            curr_loss_rr = loss_func(X_train, y_train, weightsRR)

        curr_coord = (curr_coord + 1) % (nfeats + 1)

        loss.append(curr_loss)
        lossRandomUpdate.append(curr_loss_rand_update)
        lossMaxGradUpdate.append(curr_loss_max_grad)
        lossRR.append(curr_loss_rr)

        if e % 499 == 0 and i==0:
            print(
                f"Epoch {e} :: Current Loss: {curr_loss:0.2f} :: {curr_loss_rand_update:0.2f} :: {curr_loss_max_grad:0.2f} :: {curr_loss_rr:0.2f}"
            )

    fin_loss.append(
        (curr_loss, curr_loss_rand_update, curr_loss_max_grad, curr_loss_rr)
    )

    fig.add_trace(
        go.Scatter(
            x=list(range(epochs)), y=loss, mode="lines", name=f"All Weights Update"
        )
    )

    fig.add_trace(
        go.Scatter(
            x=list(range(epochs)),
            y=lossRandomUpdate,
            mode="lines",
            name=f"Random Weights Update",
        )
    )

    fig.add_trace(
        go.Scatter(
            x=list(range(epochs)),
            y=lossMaxGradUpdate,
            mode="lines",
            name=f"Max Gradient Weight Update",
        )
    )

    fig.add_trace(
        go.Scatter(
            x=list(range(epochs)),
            y=lossRR,
            mode="lines",
            name=f"Round Robin Weight Update",
        )
    )

fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    legend=dict(yanchor="top", y=0.98, xanchor="right", x=0.99),
)

fig.update_xaxes(title_text="Epoch", title_standoff=5)
fig.update_yaxes(title_text="Loss  (log-scale)", title_standoff=5,type="log")
fig.show()


Epoch 0 :: Current Loss: 10.77 :: 89.38 :: 33.05 :: 253.11
Epoch 499 :: Current Loss: 0.08 :: 0.75 :: 0.35 :: 0.67


In [ ]:
# Final Loss Graphs 
leg = ["All","Random","Max Gradient","Round Robin"]

df=pd.DataFrame(fin_loss)
df.columns=leg
df=df.reset_index()
df=pd.melt(df,id_vars=["index"],value_vars=leg)
df=df.rename(columns={"value":"Final Loss","index":"#Runs","variable":"Weight Update Strategies"})

# Final Loss vs No. of Runs Plot
fig=px.line(df,x="#Runs",y="Final Loss",color="Weight Update Strategies",height=600,width=800)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.show()


In [ ]:
# Final Loss Distribution Plot
fig = px.histogram(df,x="Final Loss",color="Weight Update Strategies",height=500,width=800,nbins=50,barmode="overlay",marginal="box")
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.show()

In [ ]:
print("Mean Final Loss (with 95% confidence interval): ")
for i,l in enumerate(leg):
    print(f"{l} Weight Update Strategy: \n {np.mean(fin_loss,0)[i]:0.2f} +- {2*np.std(fin_loss,0)[i]:0.2f}   ")